Installing the nessicary libraries and setting up the envorenment 

In [ ]:
!pip install boto3

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.0.2-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
#seting up the environment to run Pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
#importing the libraries
import boto3
import json

In [ ]:
#Creating a session
session = boto3.Session(
    aws_access_key_id = 'AWS_ACCESS_KEY',
    aws_secret_access_key = 'AWS_SECRET_KEY'
)

In [ ]:
#connecting to Amazon s3 Bucket
s3 = session.resource('s3')

In [ ]:
bucket = s3.Bucket('BUCKET NAME')

In [ ]:
#checking bucket for all objects
for item in bucket.objects.all():
    print(item)

In [ ]:
#creating the client to get the JSON files
client = boto3.client('s3',
    aws_access_key_id = 'AWS_ACCESS_KEY',
    aws_secret_access_key = 'AWS_SECRET_KEY'
)

In [ ]:
obj_1 = client.get_object(Bucket = 'BUCKET NAME', Key = '1.jl')
obj_2 = client.get_object(Bucket = 'BUCKET NAME', Key = '2.jl')

In [ ]:
item1 = obj_1['Body'].read().decode()

In [ ]:
item1 = item1.split("\n")

In [ ]:
len(item1)

In [ ]:
#converting strings into json files
json_list = []

for item in split:
  try:
    obj = json.loads(item)
    json_list.append(obj)
  except json.decoder.JSONDecodeError:
      print(item, " String could not be converted to JSON")


In [ ]:
len(json_list)

In [ ]:
#returning values for category_id and quantity

for i in item1:
  if i == 'category_id':
    print(item1[i])
  if i == 'quantity':
    print(item1[i])

In [ ]:
#returnign values for category_id and quantity
agg_dict = {}

for item in json_list:
  for i in item:
    if i == 'item_id':
      key = item[i]
    if i == 'category_id':
      category = item[i]
    if i == 'quantity':
      qty = item[i]
      agg_dict[key] = [category,qty]

In [ ]:
#spot check agg_dict
item1 = json_list[7000]
item1

In [ ]:
agg_dict['29358354']

In [ ]:
#returnign values for category_id and quantity list for pyspark df
agg_list = []

for item in json_list:
  obj_dict = {}
  for i in item:
    if i == 'item_id':
      item_id = item[i]
      obj_dict['item_id'] = item_id
    if i == 'category_id':
      category = item[i]
      obj_dict['category'] = category
    if i == 'quantity':
      qty = item[i]
      obj_dict['quantity'] = qty
  agg_list.append(obj_dict)

creating functions for the steps of the cleaning process

In [ ]:
#Method for API to JSON object from multi JASON request string 
def apiObjectToJSON(obj):
  objStr = obj['Body'].read().decode()
  split = test.split("\n")

  json_list = []

  for item in split:
    try:
      obj = json.loads(item)
      json_list.append(obj)
    except json.decoder.JSONDecodeError:
      print(item, " String could not be converted to JSON")

  return json_list

In [ ]:
#Method for JSON to dictonary with item_id, category_id, quantity
def jsonListToDict(json_list):
  agg_list = []

  for item in json_list:
    obj_dict = {}
    for i in item:
      if i == 'item_id':
        item_id = item[i]
        obj_dict['item_id'] = item_id
      if i == 'category_id':
        category = item[i]
        obj_dict['category'] = category
      if i == 'quantity':
        qty = item[i]
        obj_dict['quantity'] = qty
    agg_list.append(obj_dict)
    
  return agg_list

In [ ]:
#used cleaning methods on second sample
json_obj_2 = apiObjectToJSON(obj_2)

agg_list_obj2 = jsonListToDict(json_obj_2)


In [ ]:
#converting dictionary into spark sql dataframe
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
df = spark.createDataFrame(agg_list)
print(df.schema)
df.show()

In [ ]:
df.createOrReplaceTempView("json_1")

In [ ]:
sql_agg = spark.sql("SELECT category AS category_id, SUM(quantity) AS total FROM json_1 GROUP BY category")

In [ ]:
sql_agg.show()

In [ ]:
#test calibration
sql_agg_test = spark.sql("SELECT category AS category_id, SUM(quantity) AS total FROM json_1 WHERE category = '976760_3788935' GROUP BY category")
sql_agg_test.show()

In [ ]:
sql_agg.coalesce(1).write.format('json').save('/1-total.json')

In [ ]:
df2 = spark.createDataFrame(agg_list_obj22)
print(df.schema)
df.show()

In [ ]:
df2.createOrReplaceTempView("json_2")

In [ ]:
sql_agg2 = spark.sql("SELECT category AS category_id, SUM(quantity) AS total FROM json_2 GROUP BY category")
sql_agg2.show()

In [ ]:
#2test calibration
sql_agg_test2 = spark.sql("SELECT category AS category_id, SUM(quantity) AS total FROM json_2 WHERE category = '976760_3788935_2118445' GROUP BY category")
sql_agg_test2.show()

In [ ]:
sql_agg2.coalesce(1).write.format('json').save('/2-total.json')